# Data exploration


In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
from kego.constants import PATH_DATA
from kego.files.files import list_files
import kego.plotting
import obonet
import pandas as pd
import polars as pl
import re
from Bio import SeqIO

In [ ]:
list(list_files(PATH_DATA, return_absolute_path=True))

In [ ]:
!head -3  '/home/kristian/projects/kego/data/cafa/cafa-6-protein-function-prediction/sample_submission.tsv'

In [ ]:
!head -10 '/home/kristian/projects/kego/data/cafa/cafa-6-protein-function-prediction/Train/train_sequences.fasta'

In [ ]:
!head  '/home/kristian/projects/kego/data/cafa/cafa-6-protein-function-prediction/Train/train_terms.tsv'

In [ ]:
def load_fasta(filepath):
    with open(filepath) as fp:
        records = [
            {
                "database": record.name.split("|")[0],
                "EntryID": record.name.split("|")[1],
                "gene_name": record.name.split("|")[2],
                "sequence": str(record.seq),
            }
            for record in SeqIO.parse(fp, "fasta")
        ]

    df = pl.DataFrame(records)
    return df


sequences = load_fasta(
    "/home/kristian/projects/kego/data/cafa/cafa-6-protein-function-prediction/Train/train_sequences.fasta"
)
sequences

In [ ]:
terms = pl.read_csv(
    "/home/kristian/projects/kego/data/cafa/cafa-6-protein-function-prediction/Train/train_terms.tsv",
    separator="\t",
)
terms

In [ ]:
train = sequences.join(terms, on="EntryID")
train

In [ ]:
print(
    f"There are many terms ({len(train["term"].unique())}) compared to total number of unique proteins ({len(train["sequence"].unique())})"
)

In [ ]:
# as expected https://www.uniprot.org/uniprotkb/A0JNW5/entry
len(
    sequences.filter(pl.col("EntryID") == "A0JNW5")["sequence"]
    .to_numpy()
    .item()
)

In [ ]:
_ = kego.plotting.plot_histogram(
    "sequence",
    df=train,
    font_size=12,
    log=["false", "log"],
    title="Counts of sequences with that many terms",
    label_x="Terms available in sequence",
    label_y="# of sequences",
)

In [ ]:
train

In [ ]:
!head '/home/kristian/projects/kego/data/cafa/cafa-6-protein-function-prediction/Train/train_taxonomy.tsv'

In [ ]:
taxonomy = pl.read_csv(
    "/home/kristian/projects/kego/data/cafa/cafa-6-protein-function-prediction/Train/train_taxonomy.tsv",
    has_header=False,
    separator="\t",
    new_columns=["EntryID", "TaxonID"],
)
taxonomy = taxonomy.with_columns(pl.col("TaxonID").cast(pl.String))
taxonomy

In [ ]:
train = train.join(taxonomy, on="EntryID")
train

In [ ]:
kego.plotting.plot_histogram("TaxonID", df=train, log=("log", "false"))

In [ ]:
!head '/home/kristian/projects/kego/data/cafa/cafa-6-protein-function-prediction/IA.tsv'

In [ ]:
ia = pl.read_csv(
    "/home/kristian/projects/kego/data/cafa/cafa-6-protein-function-prediction/IA.tsv",
    separator="\t",
    has_header=False,
    new_columns=["EntryID", "weight"],
)
ia

In [ ]:
train.write_csv("train.csv")

In [ ]:
ia.write_csv("ia.csv")

In [ ]:
!head '/home/kristian/projects/kego/data/cafa/cafa-6-protein-function-prediction/Test/testsuperset.fasta'

In [ ]:
def load_test(filepath):
    with open(filepath) as fp:
        records = [
            {
                "EntryID": record.description.split(" ")[0],
                "TaxonID": record.description.split(" ")[1],
                "sequence": str(record.seq),
            }
            for record in SeqIO.parse(fp, "fasta")
        ]

    df = pl.DataFrame(records)
    return df


test = load_test(
    "/home/kristian/projects/kego/data/cafa/cafa-6-protein-function-prediction/Test/testsuperset.fasta"
)

In [ ]:
test.write_csv("test.csv")

In [ ]:
224_309 * 1500